# Function Lib

In [1]:
%reset -f 
import plotly.express as px
import plotly.graph_objects as go
import time
from pathlib import Path
import pandas as pd
import numpy as np
import os

result_df = pd.DataFrame()
result_df_ip = pd.DataFrame()
plot_df_caller = pd.DataFrame()

global rep
global df

def open_file_nf_6pro_3ch_rasp_ff(file_name):
    """
    Reads a CSV file for reels video experiment with the new format.
    Calculates energy consumption for video watching sessions using both original and optimized methods.
    
    New format includes:
    - Proper datetime timestamps
    - Pre-calculated accumulated energy (acc_BAT_Wh, acc_BB_Wh, acc_PA_Wh)
    - acc_samples_total instead of count
    
    Returns:
        pd.DataFrame: Processed DataFrame with timestamps, SPS, and energy data.
        float: Mean SPS.
        float: Count-based mean SPS.
        float: Log duration in seconds.
        dict: Energy calculations (original method).
        dict: Energy calculations (optimized method).
    """
    # Read CSV file
    df = pd.read_csv(file_name)
    
    # Parse the combined voltage/current/power column
    power_data = df['V_BAT,I_BAT,P_BAT,V_BB,I_BB,P_BB,V_PA,I_PA,P_PA'].str.split(',', expand=True)
    power_data.columns = ['V_BAT', 'I_BAT', 'P_BAT', 'V_BB', 'I_BB', 'P_BB', 'V_PA', 'I_PA', 'P_PA']
    
    # Convert to numeric
    power_data = power_data.apply(pd.to_numeric, errors='coerce')
    
    # Add parsed columns to dataframe
    df = pd.concat([df, power_data], axis=1)
    
    # Convert timestamp to datetime
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    
    # Calculate time differences in seconds
    df['dt'] = df['Timestamp'].diff().dt.total_seconds().fillna(0)
    
    # Calculate RF power (BB + PA)
    df['P_RF'] = df['P_BB'] + df['P_PA']
    
    # =================== ORIGINAL METHOD (Trapezoidal Integration) ===================
    # Initialize energy columns for original method
    df['E_BAT_orig'] = 0.0
    df['E_RF_orig'] = 0.0
    df['E_PA_orig'] = 0.0
    df['E_BB_orig'] = 0.0
    
    # Calculate cumulative energy using trapezoidal rule (original method)
    for i in range(1, len(df)):
        if df.at[i, 'dt'] > 0:  # Only calculate if time difference is positive
            # Battery energy
            df.at[i, 'E_BAT_orig'] = df.at[i-1, 'E_BAT_orig'] + np.trapz(
                [df.at[i-1, 'P_BAT'], df.at[i, 'P_BAT']], 
                x=[0, df.at[i, 'dt']]
            )
            
            # RF energy
            df.at[i, 'E_RF_orig'] = df.at[i-1, 'E_RF_orig'] + np.trapz(
                [df.at[i-1, 'P_RF'], df.at[i, 'P_RF']], 
                x=[0, df.at[i, 'dt']]
            )
            
            # PA energy
            df.at[i, 'E_PA_orig'] = df.at[i-1, 'E_PA_orig'] + np.trapz(
                [df.at[i-1, 'P_PA'], df.at[i, 'P_PA']], 
                x=[0, df.at[i, 'dt']]
            )
            
            # BB energy
            df.at[i, 'E_BB_orig'] = df.at[i-1, 'E_BB_orig'] + np.trapz(
                [df.at[i-1, 'P_BB'], df.at[i, 'P_BB']], 
                x=[0, df.at[i, 'dt']]
            )
        else:
            # Copy previous values if no time difference
            df.at[i, 'E_BAT_orig'] = df.at[i-1, 'E_BAT_orig']
            df.at[i, 'E_RF_orig'] = df.at[i-1, 'E_RF_orig']
            df.at[i, 'E_PA_orig'] = df.at[i-1, 'E_PA_orig']
            df.at[i, 'E_BB_orig'] = df.at[i-1, 'E_BB_orig']
    
    # =================== OPTIMIZED METHOD (Pre-calculated Values) ===================
    # Convert accumulated energy from Wh to Joules (1 Wh = 3600 J)
    # df['E_BAT_opt'] = df['acc_BAT_Wh'] * 3600
    # df['E_BB_opt'] = df['acc_BB_Wh'] * 3600
    # df['E_PA_opt'] = df['acc_PA_Wh'] * 3600
    # df['E_RF_opt'] = (df['acc_BB_Wh'] + df['acc_PA_Wh']) * 3600
    
    # =================== SPS CALCULATIONS ===================
    # Calculate SPS using sample count differences
    df['sample_diff'] = df['acc_samples_total'].diff().fillna(0)
    df['SPS'] = np.where(df['dt'] > 0, df['sample_diff'] / df['dt'], 0)
    
    # Calculate mean SPS (excluding zeros and invalid values)
    valid_sps = df['SPS'][(df['SPS'] > 0) & (df['dt'] > 0)]
    sps_mean = valid_sps.mean() if len(valid_sps) > 0 else 0
    
    # Alternative SPS calculation using time windows
    df['time_second'] = df['Timestamp'].dt.floor('s')
    sps_by_second = df.groupby('time_second')['sample_diff'].sum()
    sps_count_mean = sps_by_second.mean() if len(sps_by_second) > 0 else 0
    
    # =================== TIME FORMATTING ===================
    df['time_sec_abs'] = (df['Timestamp'] - df['Timestamp'].min()).dt.total_seconds()
    df['minutes'], df['seconds'] = divmod(df['time_sec_abs'], 60)
    df['seconds'], df['milliseconds'] = divmod(df['seconds'], 1)
    df['milliseconds'] *= 1000
    df['time_formated_abs'] = (df['minutes'].astype(int).astype(str).str.zfill(2) + ':' + 
                              df['seconds'].astype(int).astype(str).str.zfill(2) + '.' + 
                              df['milliseconds'].astype(int).astype(str).str.zfill(3))
    
    # Calculate log duration
    log_duration = (df['Timestamp'].max() - df['Timestamp'].min()).total_seconds()
    
    # =================== VIDEO WATCHING PHASES ===================
    # Process useful_data for video watching phases (if applicable)
    if 'useful_data' in df.columns:
        # Create groups based on useful_data changes (video watching periods)
        df['video_session'] = (df['useful_data'].diff() != 0).cumsum()
        df['is_watching'] = df['useful_data'] == 1
    else:
        # If no useful_data column, consider entire session as watching
        df['video_session'] = 1
        df['is_watching'] = True
    
    # =================== ENERGY CALCULATIONS FOR BOTH METHODS ===================
    # Original method - total accumulated energy
    energy_orig = {
        'total_E_BAT': df['E_BAT_orig'].iloc[-1] if len(df) > 0 else 0,
        'total_E_RF': df['E_RF_orig'].iloc[-1] if len(df) > 0 else 0,
        'total_E_PA': df['E_PA_orig'].iloc[-1] if len(df) > 0 else 0,
        'total_E_BB': df['E_BB_orig'].iloc[-1] if len(df) > 0 else 0
    }
    

    
    return df, sps_mean, sps_count_mean, log_duration, energy_orig


def dataset_analyze_rasp_ff(file_name):
    """
    Analyzes reels video experiment data and assembles results in a standardized table.
    Supports both static and dynamic experiment conditions.
    
    Filename formats:
    Static: exp_total_device_ran_platform_condition_sps.csv
    Dynamic: exp_total_device_ran_platform_condition_path_from_to_sps.csv
    """
    global result_df, section_df, duplicate_rows_result_df
    
    # Process the file
    df, sps, sps_count, duration, energy_orig = open_file_nf_6pro_3ch_rasp_ff(file_name)
    
    section_df = df
    
    # Parse filename components
    file_name_base = os.path.basename(file_name)
    filename_parts = file_name_base.replace('.csv', '').split('_')
    
    # Extract basic experiment info
    exp_number = filename_parts[0] if len(filename_parts) >= 1 else None
    total_exp = filename_parts[1] if len(filename_parts) >= 2 else None
    device = filename_parts[2] if len(filename_parts) >= 3 else None
    ran_tech = filename_parts[3] if len(filename_parts) >= 4 else None
    platform = filename_parts[4] if len(filename_parts) >= 5 else None
    condition = filename_parts[5] if len(filename_parts) >= 6 else None
    
    # Handle dynamic vs static experiments
    if condition and condition.lower() == 'dyna':
        # Dynamic experiment: exp_total_device_ran_platform_dyna_path_from_to_sps
        path = filename_parts[6] if len(filename_parts) >= 7 else None
        from_location = filename_parts[7] if len(filename_parts) >= 8 else None
        to_location = filename_parts[8] if len(filename_parts) >= 9 else None
        sps_info = filename_parts[9] if len(filename_parts) >= 10 else None
        experiment_type = "Dynamic"
        location_info = f"{from_location} to {to_location}" if from_location and to_location else "N/A"
        path_info = path if path else "N/A"
    else:
        # Static experiment: exp_total_device_ran_platform_stat_sps
        sps_info = filename_parts[6] if len(filename_parts) >= 7 else None
        experiment_type = "Static"
        location_info = "Fixed Location"
        path_info = "N/A"
    
    # Extract SPS value from sps_info (remove 'sps' suffix)
    sps_value = sps_info.replace('sps', '') if sps_info else None
    
    # Calculate duration in minutes for energy per minute calculations
    duration_minutes = duration / 60 if duration > 0 else 1  # Avoid division by zero
    
    # Helper function to safely format values
    def safe_format(value, format_str='{:.2f}'):
        return format_str.format(value) if not pd.isna(value) and value != 0 else '0.00'
    
    def safe_format_percent(value):
        return '{:.2f}%'.format(value) if not pd.isna(value) else 'N/A'
    
    # Calculate percentages with safety checks
    RF_percent_bat_orig = (energy_orig['total_E_RF'] / energy_orig['total_E_BAT'] * 100) if energy_orig['total_E_BAT'] != 0 else np.nan
  
    # Create result row
    row = {
        # Experiment Information
        'File name': file_name_base,
        'Exp Number': exp_number,
        'Total Experiments': total_exp,
        'Device': device,
        'RAN Technology': ran_tech,
        'Platform': platform,
        'Condition': condition,
        'Path': path_info,
        'Location Route': location_info,
        
        # Energy per Minute - Original Method (Trapezoidal Integration)
        'E_RF Jm': safe_format(energy_orig['total_E_RF'] / duration_minutes),
        'E_BAT Jm': safe_format(energy_orig['total_E_BAT'] / duration_minutes),

        'E_BB Jm': safe_format(energy_orig['total_E_BB'] / duration_minutes),
        'E_PA Jm': safe_format(energy_orig['total_E_PA'] / duration_minutes),

        

        
        # Total Energy - Original Method
        'Total E_RF J': safe_format(energy_orig['total_E_RF']),
        'Total E_BAT J': safe_format(energy_orig['total_E_BAT']),

        'Total E_PA J': safe_format(energy_orig['total_E_PA']),
        'Total E_BB J': safe_format(energy_orig['total_E_BB']),
        
        # Percentages - Original Method
        'E_RF % BAT': safe_format_percent(RF_percent_bat_orig),

        

        # Session Information
        'Total Duration (sec)': safe_format(duration),
        'Total Duration (min)': safe_format(duration_minutes),
        'Measured SPS': '{:.5f}'.format(sps) if not pd.isna(sps) else '0.00000',
        'SPS Count Method': '{:.5f}'.format(sps_count) if not pd.isna(sps_count) else '0.00000',
        
    }
    
    # Add to result dataframe
    result_df = pd.concat([result_df, pd.DataFrame([row])], ignore_index=True)
    duplicate_rows_result_df = result_df[result_df.duplicated()]
    
    return duration, energy_orig, section_df, sps


def apply_exponential_moving_average(data, span):
    """
    Apply exponential moving average to smooth data.
    
    Args:
        data: pandas Series or array-like data
        span: int, span for the exponential moving average
        
    Returns:
        pandas Series: smoothed data
    """
    return data.ewm(span=span, adjust=False).mean()

# Initialize global variables if they don't exist
try:
    result_df
except NameError:
    result_df = pd.DataFrame()

try:
    section_df
except NameError:
    section_df = pd.DataFrame()

try:
    duplicate_rows_result_df
except NameError:
    duplicate_rows_result_df = pd.DataFrame()

In [2]:

def plot_large_dataset(file_path, chunk_size=1000000, downsample_method='uniform', n_points=10000):
    """
    Plot Reading vs Seconds from a large dataset by processing it in chunks and downsampling.
    
    Parameters:
    -----------
    file_path : str
        Path to the data file
    chunk_size : int
        Number of rows to read in each chunk
    downsample_method : str
        Method for downsampling: 'uniform', 'mean', or 'bin'
    n_points : int
        Target number of points to plot
    """
    print(f"Starting to process dataset: {Path(file_path).name}")
    start_time = time.time()
    
    # First, determine the header structure and column names
    print("Examining file structure...")
    with open(file_path, 'r') as f:
        # Read first few lines to analyze header
        header_lines = [f.readline() for _ in range(20)]  # Read up to 20 lines to find header
    
    # Look for the line that seems to contain column names
    header_row = None
    for i, line in enumerate(header_lines):
        if 'Index' in line and ('Second' in line or 'Time' in line):
            header_row = i
            print(f"Found header at line {header_row}: {line.strip()}")
            header_content = line.strip()
            break
    
    if header_row is None:
        # If we couldn't find the header, assume it's the last non-empty line before data
        for i in reversed(range(len(header_lines))):
            if header_lines[i].strip():
                header_row = i
                header_content = header_lines[i].strip()
                print(f"Using line {header_row} as header: {header_content}")
                break
    
    # Determine column names from the header
    columns = [col.strip() for col in header_content.split(',')]
    print(f"Detected columns: {columns}")
    
    # Find appropriate column names for time and reading
    time_col = None
    reading_col = None
    
    # Look for time/seconds column
    time_candidates = ['Seconds', 'Time', 'seconds', 'time']
    for col in columns:
        if col in time_candidates or any(tc in col for tc in time_candidates):
            time_col = col
            break
    
    # Look for reading column
    reading_candidates = ['Reading', 'Value', 'reading', 'value', 'Data']
    for col in columns:
        if col in reading_candidates or any(rc in col for rc in reading_candidates):
            reading_col = col
            break
    
    # If we couldn't find matching columns, use Index (first column) and the second column
    if time_col is None:
        if 'Index' in columns:
            time_col = 'Index'
        else:
            time_col = columns[0]
        print(f"Using '{time_col}' as time column")
    
    if reading_col is None:
        # Use the second column as reading if different from time column
        for col in columns:
            if col != time_col:
                reading_col = col
                break
        if reading_col is None and len(columns) > 1:
            reading_col = columns[1]
        print(f"Using '{reading_col}' as reading column")
    
    print(f"Selected columns: Time = '{time_col}', Reading = '{reading_col}'")
    
    # First, let's find the min and max of time values to determine the range
    min_seconds = float('inf')
    max_seconds = float('-inf')
    
    print("Scanning file for time range...")
    chunks_read = 0
    total_rows = 0
    
    # Read file in chunks to determine time range
    for chunk in pd.read_csv(file_path, skiprows=header_row, chunksize=chunk_size, 
                            usecols=[time_col, reading_col]):
        min_seconds = min(min_seconds, chunk[time_col].min())
        max_seconds = max(max_seconds, chunk[time_col].max())
        chunks_read += 1
        total_rows += len(chunk)
        print(f"Scanned chunk {chunks_read}, total rows: {total_rows}")
    
    print(f"Time range: {min_seconds/60:.2f} to {max_seconds/60:.2f} minutes ({min_seconds:.2f} to {max_seconds:.2f} seconds)")
    
    # For binning approach
    if downsample_method == 'bin':
        # Create bins for time ranges
        num_bins = n_points
        bin_edges = np.linspace(min_seconds, max_seconds, num_bins + 1)
        bin_width = (max_seconds - min_seconds) / num_bins
        
        # Arrays to store results
        bin_counts = np.zeros(num_bins)
        bin_sums = np.zeros(num_bins)
        
        print("Processing chunks for binning...")
        chunks_read = 0
        
        # Bin centers for plotting - calculate here but use later
        bin_centers = min_seconds + (np.arange(num_bins) + 0.5) * bin_width
        
        # Process each chunk
        for chunk in pd.read_csv(file_path, skiprows=header_row, chunksize=chunk_size,
                                usecols=[time_col, reading_col]):
            # Assign each row to a bin
            bin_indices = np.floor((chunk[time_col] - min_seconds) / bin_width).astype(int)
            # Handle edge case
            bin_indices = np.clip(bin_indices, 0, num_bins - 1)
            
            # Update bin counts and sums for this chunk
            for i in range(num_bins):
                mask = (bin_indices == i)
                bin_counts[i] += mask.sum()
                bin_sums[i] += chunk.loc[mask, reading_col].sum()
            
            chunks_read += 1
            print(f"Processed chunk {chunks_read} for binning")
        
        # Calculate mean for each bin
        bin_means = np.zeros(num_bins)
        for i in range(num_bins):
            if bin_counts[i] > 0:
                bin_means[i] = bin_sums[i] / bin_counts[i]
        
        # Bin centers for plotting - convert to minutes
        bin_centers_min = bin_centers / 60.0
        
        # Create Plotly figure
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=bin_centers_min,
            y=bin_means,
            mode='lines',
            line=dict(width=2),
            name=f'{reading_col} (Binned Average)',
            hovertemplate=f'<b>Time:</b> %{{x:.2f}} minutes<br><b>{reading_col}:</b> %{{y:.4f}}<extra></extra>'
        ))
        
        fig.update_layout(
            title=f'{reading_col} vs {time_col} - Binned Average ({n_points} bins)',
            xaxis_title=f'Minutes',
            yaxis_title=f'{reading_col} (Average)',
            width=1200,
            height=600,
            showlegend=True,
            hovermode='x unified'
        )
        
        fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
        fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
        
    elif downsample_method == 'uniform':
        # Uniform sampling - take evenly spaced chunks
        seconds_all = []
        readings_all = []
        
        # Calculate how many rows to skip between samples
        total_rows_estimate = 20000000  # From your header info
        skip_factor = max(1, total_rows_estimate // n_points)
        
        print(f"Using uniform sampling with skip factor: {skip_factor}")
        
        # Read only the rows we need
        sampled_data = pd.read_csv(file_path, skiprows=lambda x: x == 0 or (x > header_row and x % skip_factor != 0),
                                  usecols=[time_col, reading_col])
        
        print(f"Sampled {len(sampled_data)} points from dataset")
        
        # Convert seconds to minutes for plotting
        sampled_data['time_minutes'] = sampled_data[time_col] / 60.0
        
        # Create Plotly figure
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=sampled_data['time_minutes'],
            y=sampled_data[reading_col],
            mode='lines',
            line=dict(width=1),
            name=f'{reading_col} (Uniform Sample)',
            hovertemplate=f'<b>Time:</b> %{{x:.2f}} minutes<br><b>{reading_col}:</b> %{{y:.4f}}<extra></extra>'
        ))
        
        fig.update_layout(
            title=f'{reading_col} vs {time_col} - Uniform Sampling (approx. {n_points} points)',
            xaxis_title=f'{time_col} (minutes)',
            yaxis_title=reading_col,
            width=1200,
            height=600,
            showlegend=True,
            hovermode='x unified'
        )
        
        fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
        fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
    
    elif downsample_method == 'mean':
        # Determine the number of chunks to process
        chunk_means = []
        chunk_times = []
        
        print("Processing chunks for mean values...")
        chunks_read = 0
        
        # Process each chunk
        for chunk in pd.read_csv(file_path, skiprows=header_row, chunksize=chunk_size,
                                usecols=[time_col, reading_col]):
            # Calculate mean for this chunk
            mean_reading = chunk[reading_col].mean()
            mean_time = chunk[time_col].mean()
            
            chunk_means.append(mean_reading)
            chunk_times.append(mean_time)
            
            chunks_read += 1
            print(f"Processed chunk {chunks_read} for means")
        
        # Convert seconds to minutes for plotting
        chunk_times_min = [t / 60.0 for t in chunk_times]
        
        # Create Plotly figure
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=chunk_times_min,
            y=chunk_means,
            mode='lines+markers',
            line=dict(width=2),
            marker=dict(size=6),
            name=f'{reading_col} (Chunk Averages)',
            hovertemplate=f'<b>Time:</b> %{{x:.2f}} minutes<br><b>{reading_col}:</b> %{{y:.4f}}<extra></extra>'
        ))
        
        fig.update_layout(
            title=f'{reading_col} vs {time_col} - Chunk Averages ({chunks_read} chunks)',
            xaxis_title=f'{time_col} (minutes)',
            yaxis_title=f'{reading_col} (Average)',
            width=1200,
            height=600,
            showlegend=True,
            hovermode='x unified'
        )
        
        fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
        fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
    
    end_time = time.time()
    print(f"Processing completed in {end_time - start_time:.2f} seconds")
    
    # Save the plot as HTML (interactive) and optionally as PNG
    output_file_html = f"{reading_col}_vs_{time_col}_minutes_{downsample_method}.html"
    output_file_png = f"{reading_col}_vs_{time_col}_minutes_{downsample_method}.png"
    
    # Save as interactive HTML
    fig.write_html(output_file_html)
    print(f"Interactive plot saved as {output_file_html}")
    
    # Optionally save as PNG (requires kaleido: pip install kaleido)
    try:
        fig.write_image(output_file_png, width=1200, height=600, scale=2)
        print(f"Static plot saved as {output_file_png}")
    except Exception as e:
        print(f"Could not save PNG (install kaleido for PNG export): {e}")
    
    # Show the plot
    fig.show()
    
    return output_file_html, fig


In [3]:
from tqdm.notebook import tqdm  # Use tqdm.notebook for Jupyter environments

result_df = pd.DataFrame()
unique_filenames = set()
directory_path = './data/Experiment_Data/SIR_Experiment/Reels/'
#directory_path = 'G:\My Drive\Phd\Data\Data files\Experiment_Data\SIR_Experiment\Reels'
file_list = [ 
    os.path.join(directory_path, filename) for filename in [
            "1_5_6pro_LTE_tiktok_Dyna_T1_Belcombe_Auditorium_64sps.csv",
            "1_5_6pro_LTE_tiktok_Dyna_T1_Doua_Auditorium_64sps.csv",
            "1_5_6pro_LTE_insta_Dyna_T1_Prefecture_INSA_64sps.csv",
            "2_5_6pro_LTE_insta_stat_64sps.csv",
            "3_5_6pro_LTE_insta_Dyna_T1_Prefecture_INSA_64sps.csv",
            "3_5_6pro_LTE_tiktok_Dyna_T1_INSA_Prefecture_64sps.csv",
            "2_5_6pro_3G_tiktok_stat_64sps.csv",
            "3_5_6pro_3G_tiktok_stat_64sps.csv",
            "1_5_6pro_3G_YTshorts_stat_64sps.csv",
            "2_5_6pro_3G_insta_stat_64sps.csv",
            "1_5_6pro_3G_tiktok_stat_64sps.csv",
            "2_5_6pro_LTE_insta_Dyna_T1_Prefecture_INSA_64sps.csv",
            "2_5_6pro_LTE_YTshorts_stat_64sps.csv",
            "2_5_6pro_LTE_tiktok_stat_64sps.csv",
            "2_5_6pro_3G_YTshorts_stat_64sps.csv",
            "2_5_6pro_LTE_tiktok_Dyna_T1_INSA_Prefecture_64sps.csv",
            "1_5_6pro_LTE_YTshorts_stat_64sps.csv",
            "1_5_6pro_LTE_tiktok_stat_64sps.csv",
            "1_5_6pro_LTE_insta_stat_64sps.csv",
            "1_5_6pro_3G_insta_stat_64sps.csv"





            

    ]
]


files_passed = 0
duplicates_count = 0
problematic_files = []
print(len(file_list))  # This will output 36
for file_name in file_list:
    files_passed += 1
    print(f"{os.path.basename(file_name)}. passed. Count: {files_passed}")
    dataset_analyze_rasp_ff(file_name)

20
1_5_6pro_LTE_tiktok_Dyna_T1_Belcombe_Auditorium_64sps.csv. passed. Count: 1


/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_34407/2107472292.py:68: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, 'E_BAT_orig'] = df.at[i-1, 'E_BAT_orig'] + np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_34407/2107472292.py:74: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, 'E_RF_orig'] = df.at[i-1, 'E_RF_orig'] + np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_34407/2107472292.py:80: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, 'E_PA_orig'] = df.at[i-1, 'E_PA_orig'] + np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_34407/2107472292.py:86: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or o

1_5_6pro_LTE_tiktok_Dyna_T1_Doua_Auditorium_64sps.csv. passed. Count: 2


/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_34407/2107472292.py:68: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, 'E_BAT_orig'] = df.at[i-1, 'E_BAT_orig'] + np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_34407/2107472292.py:74: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, 'E_RF_orig'] = df.at[i-1, 'E_RF_orig'] + np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_34407/2107472292.py:80: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, 'E_PA_orig'] = df.at[i-1, 'E_PA_orig'] + np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_34407/2107472292.py:86: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or o

1_5_6pro_LTE_insta_Dyna_T1_Prefecture_INSA_64sps.csv. passed. Count: 3


...

/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_34407/2107472292.py:68: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, 'E_BAT_orig'] = df.at[i-1, 'E_BAT_orig'] + np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_34407/2107472292.py:74: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, 'E_RF_orig'] = df.at[i-1, 'E_RF_orig'] + np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_34407/2107472292.py:80: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, 'E_PA_orig'] = df.at[i-1, 'E_PA_orig'] + np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_34407/2107472292.py:86: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or o

1_5_6pro_LTE_insta_stat_64sps.csv. passed. Count: 19


/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_34407/2107472292.py:68: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, 'E_BAT_orig'] = df.at[i-1, 'E_BAT_orig'] + np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_34407/2107472292.py:74: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, 'E_RF_orig'] = df.at[i-1, 'E_RF_orig'] + np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_34407/2107472292.py:80: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, 'E_PA_orig'] = df.at[i-1, 'E_PA_orig'] + np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_34407/2107472292.py:86: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or o

1_5_6pro_3G_insta_stat_64sps.csv. passed. Count: 20


/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_34407/2107472292.py:68: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, 'E_BAT_orig'] = df.at[i-1, 'E_BAT_orig'] + np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_34407/2107472292.py:74: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, 'E_RF_orig'] = df.at[i-1, 'E_RF_orig'] + np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_34407/2107472292.py:80: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, 'E_PA_orig'] = df.at[i-1, 'E_PA_orig'] + np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_34407/2107472292.py:86: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or o

In [4]:
# import pandas as pd
# import numpy as np
# import plotly.express as px
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
# import os

# Your file path
file_name = './data/Experiment_Data/SIR_Experiment/Reels/2_5_6pro_3G_YTshorts_stat_64sps.csv'

# Load and process the data using the optimized function
df1, sps_mean, sps_count_mean, duration, energy_org = open_file_nf_6pro_3ch_rasp_ff(file_name)

print(f"File loaded successfully!")
print(f"Duration: {duration:.2f} seconds ({duration/60:.2f} minutes)")
print(f"Data points: {len(df1)}")
print(f"SPS Mean: {sps_mean:.2f}")

# P_RF is already calculated in the function, but let's ensure it's correct
df1['P_RF'] = df1['P_BB'] + df1['P_PA']

# Apply exponential moving average for smoothing
with pd.option_context("mode.copy_on_write", True):
    df1['P_BAT_smooth'] = apply_exponential_moving_average(df1['P_BAT'], 30)
    df1['P_BB_smooth'] = apply_exponential_moving_average(df1['P_BB'], 30)
    df1['P_PA_smooth'] = apply_exponential_moving_average(df1['P_PA'], 30)
    df1['P_RF_smooth'] = apply_exponential_moving_average(df1['P_RF'], 30)

# Create time column for plotting (using time_sec_abs for better axis handling)
df1['time_minutes'] = df1['time_sec_abs'] / 60

# Plot 1: Main power consumption plot
fig = px.line(df1, 
              x='time_minutes', 
              y=['P_BAT_smooth', 'P_BB_smooth', 'P_PA_smooth', 'P_RF_smooth'],
              labels={
                  'value': 'Power (W)', 
                  'time_minutes': 'Time (minutes)',
                  'variable': 'Power Type'
              },
              color_discrete_sequence=['black', 'steelblue', 'red', 'green'],
              height=600,
              width=1500,
              title=f'Power Consumption - {os.path.basename(file_name)}'
              )

# Add useful_data as secondary y-axis
fig.add_trace(
    go.Scatter(
        x=df1['time_minutes'],
        y=df1['useful_data'],
        mode='lines',
        name='useful_data',
        line=dict(color='orange', width=2),
        yaxis='y2'
    )
)

# Update layout with secondary y-axis
fig.update_layout(
    xaxis=dict(
        title='<b>Time (minutes)</b>',
        tickformat='.2f'
    ),
    yaxis=dict(
        title='<b>Power (W)</b>',
        side='left'
    ),
    yaxis2=dict(
        title='<b>Useful Data</b>',
        side='right',
        overlaying='y',
        range=[0, 1.2]
    ),
    xaxis_rangeslider_visible=True,
    legend=dict(
        x=0.770, 
        y=1.0, 
        traceorder="normal",
        bgcolor='rgba(0,0,0,0)',
        font=dict(family="Times New Roman", size=20, color="black")
    ),
    font_family="Times New Roman",
    font_color="black",
    font_size=25
)

# Update range slider thickness
fig.update_xaxes(rangeslider_thickness=0.03)

# Make legend text bold
fig.for_each_trace(lambda t: t.update(name='<b>' + t.name.replace('_smooth', '') + '</b>'))

# Show and save the plot
fig.show()
fig.write_html(f"{os.path.basename(file_name).replace('.csv', '_power_plot.html')}")


/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_34407/2107472292.py:68: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, 'E_BAT_orig'] = df.at[i-1, 'E_BAT_orig'] + np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_34407/2107472292.py:74: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, 'E_RF_orig'] = df.at[i-1, 'E_RF_orig'] + np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_34407/2107472292.py:80: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, 'E_PA_orig'] = df.at[i-1, 'E_PA_orig'] + np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_34407/2107472292.py:86: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or o

File loaded successfully!
Duration: 625.66 seconds (10.43 minutes)
Data points: 39897
SPS Mean: 1023.42


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:

# file_path = "./data/Experiment_Data/Call Test/iPX_CAll_Callee_RX_4G_DMM.csv"  # Replace with your actual file path
    
#     # You can choose from: 'uniform', 'mean', or 'bin'
# plot_method = 'bin'
    
#     # Call the function with your preferred method
# output_file, figure = plot_large_dataset(file_path, downsample_method=plot_method, n_points=5000)

In [ ]:
import pandas as pd

# Load your result_df if not done already
# result_df = pd.read_csv("result_df.csv")

# Add scenario_id column
# Create scenario_id from the relevant columns
result_df['scenario_id'] = (
    result_df['Device'].astype(str).str.strip() + "_" +
    result_df['RAN Technology'].astype(str).str.strip() + "_" +
    result_df['Platform'].astype(str).str.strip() + "_" +
    result_df['Condition'].astype(str).str.strip()
)
# Compute average energy values per scenario
# Clean energy columns just in case
energy_cols = ['E_RF Jm', 'E_BAT Jm', 'E_BB Jm', 'E_PA Jm']
result_df[energy_cols] = result_df[energy_cols].apply(pd.to_numeric, errors='coerce')

# Group by scenario_id
scenario_summary_df = result_df.groupby('scenario_id')[energy_cols].mean().reset_index()

# Optional: rename for clarity
scenario_summary_df.columns = ['scenario_id', 'E_RF_Jm', 'E_BAT_Jm', 'E_BB_Jm', 'E_PA_Jm']

# Save to CSV for frontend usage
scenario_summary_df.to_csv("frontend_energy_summary.csv", index=False)


# Show preview in notebook
scenario_summary_df.head(10)


,scenario_id,E_RF_Jm,E_BAT_Jm,E_BB_Jm,E_PA_Jm
0,6pro_3G_YTshorts_stat,54.495000,155.435,42.4300,12.065000
1,6pro_3G_insta_stat,48.990000,194.390,43.3100,5.675000
2,6pro_3G_tiktok_stat,53.016667,164.240,44.9900,8.026667
3,6pro_LTE_YTshorts_stat,68.085000,-2.520,49.1550,18.930000
4,6pro_LTE_insta_Dyna,53.000000,191.860,47.5600,5.433333
5,6pro_LTE_insta_stat,72.800000,199.715,66.8500,5.945000
6,6pro_LTE_tiktok_Dyna,58.105000,176.050,50.0325,8.070000
7,6pro_LTE_tiktok_stat,57.080000,-33.500,44.7550,12.325000
